In [1]:
from PIL import Image
from PIL.ExifTags import TAGS
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import numpy as np

### Reader

In [2]:
def readFile(file_path):
    image = Image.open(file_path)
    exif = image.getexif()
    if exif is None:
        return None
    
    exif_data = {}
    for tag_id, value in exif.items():
        tag = TAGS.get(tag_id, tag_id)
        if tag == 34682:
            metadata = value
    return metadata

In [3]:
testImg = '/Users/elias/Desktop/images_to_try/SEM_image_sample_Thermo_Fisher_Helios_G4_PFIB_CXe.tif'
# mdValueIni = readFile("/Users/elias/Desktop/NFDI Tomographiedaten/20200818_AlSi13 XRM tomo2/Images/SEM Image/SEM Image - SliceImage - 001.tif")
mdValueIni = readFile(testImg)
fileMetadataList = []

print(mdValueIni)

[User]
Date=03/17/2022
Time=11:05:00 AM
User=user
UserText=HeliosPFIB
UserTextUnicode=480065006C0069006F0073005000460049004200

[System]
Type=DualBeam
Dnumber=9952707
Software=14.5.1.432
BuildNr=432
Source=FEG
Column=Elstar
FinalLens=Elstar
Chamber=xT-SDB
Stage=110 x 110
Pump=TMP
ESEM=no
Aperture=AVA
Scan=PIA 3.0
Acq=PIA 3.0
EucWD=0.004
SystemType=Helios G4 PFIB CXe
DisplayWidth=0.518
DisplayHeight=0.324

[Beam]
HV=20000
Spot=2
StigmatorX=0
StigmatorY=0
BeamShiftX=-1.60064e-007
BeamShiftY=5.87256e-006
ScanRotation=0
ImageMode=Normal
FineStageBias=
Beam=EBeam
Scan=EScan

[EBeam]
Source=FEG
ColumnType=Elstar
FinalLens=Elstar
Acq=PIA 3.0
Aperture=AVA
ApertureDiameter=6.4e-005
HV=20000
HFW=0.0004144
VFW=0.000276267
WD=0.00794789
BeamCurrent=3.2e-009
TiltCorrectionIsOn=no
DynamicFocusIsOn=no
DynamicWDIsOn=
ScanRotation=0
LensMode=Field-Free
LensModeA=
ATubeVoltage=
UseCase=
SemOpticalMode=
ImageMode=Normal
SourceTiltX=0
SourceTiltY=0
StageX=-0.00333226
StageY=-0.0115452
StageZ=0.00800001
St

In [11]:
def readMetadata(filename, text, MetadataList):
    
    mdValueIniList = text.split("\n")

    mdValueList = []

    metadataDict = {}
    indexList = []
    
    file = filename.split('/')
    
    metadataDict["filename"] = file[-1]

    for item in mdValueIniList:
            newItemitem = item.replace("\r", "")
            mdValueList.append(newItemitem)

    for item in mdValueList:       
        if "]" in item:
            index = mdValueList.index(item)
            indexList.append(index)


    for item in range(len(indexList)):
        k = item + 1

        if k == len(indexList):
            sublist = mdValueList[indexList[item]:]

        else:
            sublist = mdValueList[indexList[item]:indexList[k]]



        for item in sublist[1:]:

            if "=" in item:
                newItem = item.split('=')
        
                if newItem[1] == "":
                    newItem[1] = "NaN"
                    
                value = newItem[1]

                metadataDict[sublist[0].strip() + " " + newItem[0]] = value

    
    MetadataList.append(metadataDict)

In [5]:
import os

dirname = '/Users/elias/Desktop/NFDI Tomographiedaten/20200818_AlSi13 XRM tomo2/Images/SEM Image/'

MetadataList = []

fileNumber = 0
for fname in os.listdir(dirname):
    fileNumber += 1
    
    fpath = os.path.join(dirname, fname)
    
    mdValue = readFile(fpath)
    
    readMetadata(fpath, mdValue, MetadataList)

In [10]:
print(MetadataList)

[{'filename': 'SEM Image - SliceImage - 085.tif', '[User] Date': '08/18/2020', '[User] Time': '05:51:07 PM', '[User] User': 'user', '[User] UserText': 'NaN', '[User] UserTextUnicode': 'NaN', '[System] Type': 'DualBeam', '[System] Dnumber': '9952707', '[System] Software': '14.5.1.432', '[System] BuildNr': '432', '[System] Source': 'FEG', '[System] Column': 'Elstar', '[System] FinalLens': 'Elstar', '[System] Chamber': 'xT-SDB', '[System] Stage': '110 x 110', '[System] Pump': 'TMP', '[System] ESEM': 'no', '[System] Aperture': 'AVA', '[System] Scan': 'PIA 3.0', '[System] Acq': 'PIA 3.0', '[System] EucWD': '0.004', '[System] SystemType': 'Helios G4 PFIB CXe', '[System] DisplayWidth': '0.518', '[System] DisplayHeight': '0.324', '[Beam] HV': '15000', '[Beam] Spot': '1', '[Beam] StigmatorX': '0.0153243', '[Beam] StigmatorY': '0.00747505', '[Beam] BeamShiftX': '3.83553e-009', '[Beam] BeamShiftY': '7.39564e-006', '[Beam] ScanRotation': '0', '[Beam] ImageMode': 'Normal', '[Beam] FineStageBias': '

# Mapper (UNDER DEVELOPMENT)

Specs and data preparation ([details](https://github.com/kit-data-manager/Metadata-Schemas-for-Materials-Science/tree/main/SEM-FIB%20Tomography)):

* Acquisition metadata $\rightarrow$ get from .emxml file

* Dataset metadata $\rightarrow$ Some metadata is common to each image, identify which and extract only from first image, check against a few other images to ensure it really is the same?

* Image metadata $\rightarrow$ get from each image, just as it was for Zeiss


Therefore mapper should produce `n+2 jason metadata documents`, where `n` is the number of images in the dataset

In [7]:
# define a function which takes the metadata list and organizes it as a python dict

# md = MetadataList
# metadataDict = dict(md)

In [8]:
# function which maps each element defined in the map "sem_fib_nested_schema_map.json" to its defined value

    # Map the metadata dictionary
# try:
#     Map = AttributeMapping(metadataDict, newmapSEM, 'mappedTerms')
#     workingDict = Map.__dict__
# except:
#     print('One or more of the required parameters was not found.')